In [ ]:
from pathlib import Path
import pandas as pd

# === 경로 설정 ===
BASE = Path("/home/dw/ws_job_msislab/Golf_Project/data/good_data_3_erase_dieased_grass")

# === 이미지 확장자 ===
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".JPG", ".JPEG", ".PNG", ".BMP"}

# === 삭제된 파일 기록 ===
deleted_files = []

# === 모든 라벨 파일(.txt) 탐색 ===
for txt_file in BASE.rglob("*.txt"):
    with open(txt_file, "r") as f:
        lines = f.readlines()

    # class 2가 포함된 경우만 체크
    has_class2 = any(line.strip().startswith("2 ") or line.strip() == "2" for line in lines)

    if has_class2:
        # 라벨 삭제
        txt_file.unlink()
        deleted_files.append(str(txt_file))

        # 같은 이름의 이미지 파일도 삭제
        for ext in IMG_EXTS:
            img_path = txt_file.with_suffix(ext)
            if img_path.exists():
                img_path.unlink()
                deleted_files.append(str(img_path))

# === 결과 요약 ===
df = pd.DataFrame(deleted_files, columns=["Deleted Files"])
if df.empty:
    print("⚠️ class 2(diseased_grass) 라벨이 포함된 파일이 발견되지 않았습니다.")
else:
    print(f"✅ 삭제된 파일 개수: {len(df)}")
df


In [ ]:
from pathlib import Path
from collections import Counter
import pandas as pd

# === 경로 설정 ===
BASE = Path("/home/dw/ws_job_msislab/Golf_Project/data/good_data_3_erase_dieased_grass")

# === 이미지 확장자 ===
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".JPG", ".JPEG", ".PNG", ".BMP"}

# === 집계 변수 ===
cls_counter = Counter()
all_images = [p for p in BASE.rglob("*") if p.suffix in IMG_EXTS]
txt_files = list(BASE.rglob("*.txt"))

# === 클래스 개수 세기 ===
for txt in txt_files:
    with open(txt, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) > 0 and parts[0].isdigit():
                cls_counter[int(parts[0])] += 1

# === txt 없는 이미지 찾기 ===
txt_stems = {t.stem for t in txt_files}
img_no_label = [img for img in all_images if img.stem not in txt_stems]

# === 결과 정리 ===
results = {
    "총 이미지 개수": len(all_images),
    "총 라벨 파일 개수": len(txt_files),
    "Class 0 개수": cls_counter.get(0, 0),
    "Class 1 개수": cls_counter.get(1, 0),
    "라벨 없는 이미지 개수": len(img_no_label),
}

# === 출력 ===
df = pd.DataFrame(list(results.items()), columns=["항목", "개수"])
df



In [ ]:
# augment_balance_final_no_c2.py
# 요구사항(수정 후)
# - 목표 객체수: {0:3500, 1:3500}  (class 2는 완전히 제외)
# - 단위: 이미지+라벨 1쌍을 복사/증강할 때 그 파일 안 "객체 수"를 누적 카운트
# - 재사용 제한: 모든 파일 기본 2회
# - 라벨 없는 원본 이미지는 background로 간주: 원본당 +1장 생성(총 2배), 빈 라벨(.txt)로 저장
# - Albumentations: 골프장 조명/색감 + 약한 기하(수평반전만). 최신 파라미터로 경고 제거
# - Albumentations 호출 "직전" YOLO bbox를 [0,1]로 안전 보정(yolo_sanitize) → 경계 부동소수 오차 ValueError 방지
# - 라벨 저장 시 클래스 ID 정수 강제, 좌표 [0..1] 클리핑, 비정상 라인 제거
# - 출력 구조: BASE/aug_balanced/ (이미지와 .txt 라벨을 같은 한 폴더에 저장)
# - 진행 로그: BASE/aug_balanced/log.csv

from pathlib import Path
from collections import Counter, defaultdict
import random, shutil, csv, os
import cv2

# ====== 사용자 설정 ======
BASE = Path("/home/dw/ws_job_msislab/Golf_Project/data/good_data_3_erase_dieased_grass")
OUT_DIR = BASE / "aug_balanced"     # 단일 폴더 (이미지/라벨을 모두 저장)
IMG_EXTS = [".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG"]
TARGET = {0: 3500, 1: 3500}         # ✅ class 2 제거
SEED = 42
random.seed(SEED)

# 재사용 제한 (모두 동일)
MAX_USES_DEFAULT = 2

# 배경 증강 배수(+1이면 총 2배)
BG_AUG_MULTIPLIER = 1

# ====== 증강 설정 (최신 파라미터로 경고 제거) ======
import albumentations as A

# 색/조명 위주 + 좌우반전 (크기 변형 없음)
aug = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=(-0.12, 0.18),
                               contrast_limit=(-0.12, 0.18), p=0.9),
    A.HueSaturationValue(hue_shift_limit=2,
                         sat_shift_limit=12,
                         val_shift_limit=10, p=0.8),
    A.RGBShift(r_shift_limit=8, g_shift_limit=6, b_shift_limit=6, p=0.5),
    A.RandomGamma(gamma_limit=(90, 120), p=0.4),
    A.OneOf([
        A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.5),
        A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.08, 0.25), p=0.5),
        A.NoOp()
    ], p=0.4),
],
    bbox_params=A.BboxParams(
        format="yolo",
        label_fields=["class_labels"],
        min_visibility=0.25
    )
)

# ====== 유틸 ======
def ensure_out_dir():
    OUT_DIR.mkdir(parents=True, exist_ok=True)

def read_yolo_label(lbl: Path, *, allow_ids={0,1}):
    """YOLO 라벨 로드. allow_ids 외 클래스는 즉시 폐기"""
    boxes, cls = [], []
    if not lbl.exists():
        return boxes, cls
    with open(lbl, "r") as f:
        for line in f:
            p = line.strip().split()
            if len(p) < 5:
                continue
            # class id 정수화 시도
            try:
                c = int(float(p[0]))
            except Exception:
                continue
            if allow_ids is not None and c not in allow_ids:
                # ✅ class 2 등은 여기서 걸러냄
                continue
            # 좌표
            try:
                x, y, w, h = map(float, p[1:5])
            except Exception:
                continue
            # 1차 범위 클립
            x = min(max(x, 0.0), 1.0)
            y = min(max(y, 0.0), 1.0)
            w = min(max(w, 0.0), 1.0)
            h = min(max(h, 0.0), 1.0)
            if w <= 0.0 or h <= 0.0:
                continue
            boxes.append([x, y, w, h]); cls.append(c)
    return boxes, cls

def write_yolo_label(lbl_path: Path, boxes, cls, *, allow_ids={0,1}):
    """클래스 정수 강제, 좌표 [0..1] 클립, 비정상 라인 제거. 빈 라벨 허용."""
    lbl_path.parent.mkdir(parents=True, exist_ok=True)
    with open(lbl_path, "w") as f:
        for c, (x, y, w, h) in zip(cls, boxes):
            try:
                c_int = int(float(c))
            except Exception:
                continue
            if allow_ids is not None and c_int not in allow_ids:
                continue
            x = float(max(0.0, min(1.0, x)))
            y = float(max(0.0, min(1.0, y)))
            w = float(max(0.0, min(1.0, w)))
            h = float(max(0.0, min(1.0, h)))
            if w <= 0.0 or h <= 0.0:
                continue
            f.write(f"{c_int} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

def yolo_sanitize(boxes):
    """
    YOLO (cx,cy,w,h) → 경계 넘어간 박스는 잘라낸 뒤 재계산하여 [0,1]로 보정.
    Albumentations 사전검사(ValueError) 방지용.
    """
    eps = 1e-6
    out = []
    for x, y, w, h in boxes:
        l = x - w/2.0; r = x + w/2.0
        t = y - h/2.0; b = y + h/2.0
        l = max(0.0, min(1.0, l))
        r = max(0.0, min(1.0, r))
        t = max(0.0, min(1.0, t))
        b = max(0.0, min(1.0, b))
        if r - l <= eps or b - t <= eps:
            continue
        nx = (l + r) / 2.0
        ny = (t + b) / 2.0
        nw = (r - l)
        nh = (b - t)
        nx = min(max(nx, eps), 1.0 - eps)
        ny = min(max(ny, eps), 1.0 - eps)
        nw = min(max(nw, eps), 1.0 - eps)
        nh = min(max(nh, eps), 1.0 - eps)
        out.append([nx, ny, nw, nh])
    return out

def find_image_for_label(lbl: Path):
    stem = lbl.stem
    # 같은 폴더
    for ext in IMG_EXTS:
        p = lbl.with_suffix(ext)
        if p.exists():
            return p
    # 상위에서 images 폴더 탐색
    for parent in [lbl.parent, *lbl.parents]:
        img_dir = parent / "images"
        if img_dir.exists():
            for ext in IMG_EXTS:
                p = img_dir / f"{stem}{ext}"
                if p.exists():
                    return p
    # 전역 탐색(최후)
    for ext in IMG_EXTS:
        hits = list(BASE.rglob(f"{stem}{ext}"))
        if hits:
            return hits[0]
    return None

def current_counts(root: Path):
    cnt = Counter()
    for lbl in root.rglob("*.txt"):
        _, cls = read_yolo_label(lbl, allow_ids={0,1})
        cnt.update(cls)
    return cnt

def collect_background_images(base: Path):
    label_stems = {p.stem for p in base.rglob("*.txt")}
    bg_images = []
    for ext in IMG_EXTS:
        for img in base.rglob(f"*{ext}"):
            if img.stem not in label_stems:
                bg_images.append(img)
    return sorted(bg_images)

def score(deficit: Counter, fcnt: Counter):
    # 남은 결손에 기여하는 클래스만 가중합
    return sum(deficit[c] * fcnt.get(c, 0) for c in deficit if deficit[c] > 0)

def load_image(path: Path):
    img = cv2.imread(str(path))
    if img is None:
        raise RuntimeError(f"Failed to read image: {path}")
    return img

def next_unique_name(stem: str, used: set, tag: str):
    i = 1
    while True:
        s = f"{stem}_{tag}{i:04d}"
        if s not in used:
            used.add(s)
            return s
        i += 1

# ====== 준비 ======
ensure_out_dir()

# 1) 배경(라벨 없는 이미지) 2배 증강 (빈 라벨 저장)
bg_list = collect_background_images(BASE)
print(f"Background images: {len(bg_list)}")

used_names = set()
for bg_img in bg_list:
    try:
        img = load_image(bg_img)
    except Exception:
        continue
    for _ in range(BG_AUG_MULTIPLIER):
        out_stem = next_unique_name(bg_img.stem, used_names, tag="bg")
        out_img = OUT_DIR / f"{out_stem}{bg_img.suffix.lower()}"
        out_lbl = OUT_DIR / f"{out_stem}.txt"

        transformed = aug(image=img, bboxes=[], class_labels=[])
        aug_img = transformed["image"]
        cv2.imwrite(str(out_img), aug_img)

        # 빈 라벨 생성
        with open(out_lbl, "w") as f:
            pass

print("Background augmentation done.")

# 2) 라벨 있는 정상 짝 후보 수집 (class 2는 로드 단계에서 이미 제외)
label_files = sorted(BASE.rglob("*.txt"))
candidates = []  # (img_path, lbl_path, file_counts, stem)
skipped = 0
for lbl in label_files:
    boxes, cls_list = read_yolo_label(lbl, allow_ids={0,1})
    if not cls_list:   # 빈 라벨은 본 루프에서 제외(배경 파트에서 처리)
        continue
    img = find_image_for_label(lbl)
    if img is None or not img.exists():
        skipped += 1
        continue
    fcnt = Counter(cls_list)
    candidates.append((img, lbl, fcnt, lbl.stem))

if not candidates:
    raise SystemExit("No labeled image-label pairs found (after filtering out class 2).")
print(f"Labeled pairs: {len(candidates)}, skipped(no image match): {skipped}")

# 3) 초기 카운트(원본, class 0/1만)
base_cnt = current_counts(BASE)
print("Base counts (0/1 only):", dict(base_cnt))

# 4) 메인 루프(타깃 채우기)
cur = Counter(base_cnt)
deficit = Counter({c: max(0, TARGET[c]-cur.get(c,0)) for c in TARGET})
use_count = defaultdict(int)

log_path = OUT_DIR / "log.csv"
with open(log_path, "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["src_img","src_lbl","out_img","out_lbl",
                "add_c0","add_c1",
                "total_after_c0","total_after_c1"])

MAX_ITERS = 200000
iters = 0

while any(deficit[c] > 0 for c in TARGET) and iters < MAX_ITERS:
    iters += 1
    best = None
    best_score = 0

    for img, lbl, fcnt, stem in candidates:
        # ✅ 모든 파일 공통 재사용 제한
        if use_count[img] >= MAX_USES_DEFAULT:
            continue
        sc = score(deficit, fcnt)
        if sc > best_score:
            best_score = sc
            best = (img, lbl, fcnt, stem)

    if not best or best_score <= 0:
        print("No more useful candidates for remaining deficits. Stopping.")
        break

    img_path, lbl_path, fcnt, stem = best
    out_stem = next_unique_name(stem, used_names, tag="dup")
    out_img = OUT_DIR / f"{out_stem}{img_path.suffix.lower()}"
    out_lbl = OUT_DIR / f"{out_stem}.txt"

    # === 증강/저장 ===
    img = load_image(img_path)
    boxes0, cls0 = read_yolo_label(lbl_path, allow_ids={0,1})

    # Albumentations 호출 전, YOLO 박스 안전 보정 (경계 오차 방지)
    boxes0 = yolo_sanitize(boxes0)

    # 증강
    transformed = aug(image=img, bboxes=boxes0, class_labels=cls0)
    aug_img = transformed["image"]
    aug_boxes = transformed["bboxes"]
    aug_cls = transformed["class_labels"]

    # 2차 필터(너무 작은 박스 제거) + 정수 강제
    keep_boxes, keep_cls = [], []
    for (x, y, w, h), c in zip(aug_boxes, aug_cls):
        if w > 0 and h > 0:
            keep_boxes.append([float(x), float(y), float(w), float(h)])
            keep_cls.append(int(float(c)))

    # 변환 후에도 부족 클래스에 기여하는지 확인(불필요 저장 방지)
    test_cnt = Counter(keep_cls)
    if sum(test_cnt[c] for c in TARGET if deficit[c] > 0) <= 0 or not keep_cls:
        use_count[img_path] += 1  # 시도 카운트(원치 않으면 제거 가능)
        continue

    cv2.imwrite(str(out_img), aug_img)
    write_yolo_label(out_lbl, keep_boxes, keep_cls, allow_ids={0,1})
    _, saved_cls = read_yolo_label(out_lbl, allow_ids={0,1})
    add_cnt = Counter(saved_cls)

    # 누적/결손 갱신
    cur.update(add_cnt)
    for c in TARGET:
        deficit[c] = max(0, TARGET[c] - cur.get(c, 0))
    use_count[img_path] += 1

    # 로그
    with open(log_path, "a", newline="") as f:
        w = csv.writer(f)
        w.writerow([
            str(img_path), str(lbl_path), str(out_img), str(out_lbl),
            add_cnt.get(0, 0), add_cnt.get(1, 0),
            cur.get(0, 0), cur.get(1, 0)
        ])

    if iters % 50 == 0 or all(deficit[c]==0 for c in TARGET):
        print(f"[{iters}] cur={dict(cur)} deficit={dict(deficit)} last={out_stem}")

print("\n=== Finished ===")
print("Final counts:", dict(cur))
print("Deficit:", dict(deficit))
print(f"Saved to: {OUT_DIR}")
print(f"Log: {log_path}")


In [ ]:
from pathlib import Path
from collections import Counter
import pandas as pd

# === 경로 설정 ===
BASE = Path("/home/dw/ws_job_msislab/Golf_Project/data/good_data_3_erase_dieased_grass_aug")

# === 이미지 확장자 ===
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".JPG", ".JPEG", ".PNG", ".BMP"}

# === 집계 변수 ===
cls_counter = Counter()
all_images = [p for p in BASE.rglob("*") if p.suffix in IMG_EXTS]
txt_files = list(BASE.rglob("*.txt"))

# === 클래스 개수 세기 ===
for txt in txt_files:
    with open(txt, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) > 0 and parts[0].isdigit():
                cls_counter[int(parts[0])] += 1

# === txt 없는 이미지 찾기 ===
txt_stems = {t.stem for t in txt_files}
img_no_label = [img for img in all_images if img.stem not in txt_stems]

# === 결과 정리 ===
results = {
    "총 이미지 개수": len(all_images),
    "총 라벨 파일 개수": len(txt_files),
    "Class 0 개수": cls_counter.get(0, 0),
    "Class 1 개수": cls_counter.get(1, 0),
    "라벨 없는 이미지 개수": len(img_no_label),
}

# === 출력 ===
df = pd.DataFrame(list(results.items()), columns=["항목", "개수"])
df



In [ ]:
from pathlib import Path

# 경로 지정
BASE = Path("/home/dw/ws_job_msislab/Golf_Project/data/good_data_3_erase_dieased_grass_aug")

# 빈 txt 삭제
deleted = []
for txt in BASE.rglob("*.txt"):
    if txt.stat().st_size == 0:  # 파일 크기가 0바이트
        txt.unlink()
        deleted.append(str(txt))

print(f"✅ 삭제된 빈 txt 파일 개수: {len(deleted)}")
for f in deleted[:10]:  # 샘플 10개만 출력
    print(f)

if len(deleted) > 10:
    print("... 생략 ...")


In [ ]:
from pathlib import Path
from collections import Counter
import pandas as pd

# === 경로 설정 ===
BASE = Path("/home/dw/ws_job_msislab/Golf_Project/data/good_data_3_erase_dieased_grass_aug")

# === 이미지 확장자 ===
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".JPG", ".JPEG", ".PNG", ".BMP"}

# === 집계 변수 ===
cls_counter = Counter()
all_images = [p for p in BASE.rglob("*") if p.suffix in IMG_EXTS]
txt_files = list(BASE.rglob("*.txt"))

# === 클래스 개수 세기 ===
for txt in txt_files:
    with open(txt, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) > 0 and parts[0].isdigit():
                cls_counter[int(parts[0])] += 1

# === txt 없는 이미지 찾기 ===
txt_stems = {t.stem for t in txt_files}
img_no_label = [img for img in all_images if img.stem not in txt_stems]

# === 결과 정리 ===
results = {
    "총 이미지 개수": len(all_images),
    "총 라벨 파일 개수": len(txt_files),
    "Class 0 개수": cls_counter.get(0, 0),
    "Class 1 개수": cls_counter.get(1, 0),
    "라벨 없는 이미지 개수": len(img_no_label),
}

# === 출력 ===
df = pd.DataFrame(list(results.items()), columns=["항목", "개수"])
df


In [ ]:
from pathlib import Path
from collections import Counter

# 데이터셋 경로
BASE = Path("/home/dw/ws_job_msislab/Golf_Project/data/good_data_3_erase_dieased_grass_aug")

# 클래스 카운트
cls_counter = Counter()
label_files = list(BASE.rglob("*.txt"))

for lbl in label_files:
    with open(lbl, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 5 and parts[0].isdigit():
                cid = int(parts[0])
                cls_counter[cid] += 1

# 결과 출력
print("📊 클래스별 객체 수")
for cid, n in sorted(cls_counter.items()):
    print(f"클래스 {cid}: {n}개")

print(f"총 객체 수: {sum(cls_counter.values())}")
print(f"총 라벨 파일 수: {len(label_files)}")

In [ ]:
# split_iter_strat_with_bg_no_c2.py
from pathlib import Path
from collections import Counter, defaultdict
import random, shutil

# ====== 설정 ======
SRC = Path("/home/dw/ws_job_msislab/Golf_Project/data/good_data_3_erase_dieased_grass_aug")
DST = Path("/home/dw/ws_job_msislab/Golf_Project/data/good_data_3_erase_dieased_grass_aug_split")
IMG_EXTS = [".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG"]
RATIOS = {"train": 0.78, "val": 0.12, "test": 0.10}  # 합 1.0
LABELS = [0, 1]       # ✅ class 2 제거
BG = "bg"             # 백그라운드 라벨 이름
SEED = 42
random.seed(SEED)

# ====== 유틸 ======
def ensure_dirs():
    for s in RATIOS:
        (DST / "images" / s).mkdir(parents=True, exist_ok=True)
        (DST / "labels" / s).mkdir(parents=True, exist_ok=True)

def find_image(stem: str):
    for ext in IMG_EXTS:
        p = SRC / f"{stem}{ext}"
        if p.exists(): 
            return p
    return None

def parse_label_file_keep_01(p: Path):
    """
    txt를 읽어 0/1인 라벨만 남긴다.
    반환: (set_of_labels, filtered_lines)
    - set_of_labels: {0,1} 중 포함된 것들
    - filtered_lines: 'cid x y w h ...' 문자열 리스트(0/1만 남음)
    """
    labs = set()
    filtered_lines = []
    try:
        with open(p, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    try:
                        cid = int(float(parts[0]))
                    except:
                        continue
                    if cid in LABELS:
                        labs.add(cid)
                        filtered_lines.append(line.strip())
    except:
        pass
    return labs, filtered_lines

# ====== 1) 수집: (stem, img_path, lbl_path or None, labels_set, filtered_lines) ======
items = []
stems = set()
for ext in IMG_EXTS:
    for img in SRC.glob(f"*{ext}"):
        stems.add(img.stem)

for stem in sorted(stems):
    img = find_image(stem)
    if img is None: 
        continue
    lbl = SRC / f"{stem}.txt"
    if lbl.exists():
        labs, filtered = parse_label_file_keep_01(lbl)
        if labs:
            # ✅ 0/1만 남긴 filtered 라인을 보관 (복사 시 이걸로 새로 저장)
            items.append((stem, img, lbl, labs, filtered))
        else:
            # 빈/혹은 2만 있는 파일 → bg 취급
            items.append((stem, img, None, {BG}, []))
    else:
        # 라벨 파일 자체가 없으면 bg
        items.append((stem, img, None, {BG}, []))

if not items:
    raise SystemExit("No images found in source folder.")

all_labels = LABELS + [BG]
N = len(items)

# ====== 2) 목표치 계산 ======
target_sizes = {s: int(round(N * r)) for s, r in RATIOS.items()}
# 합 보정
while sum(target_sizes.values()) < N:
    k = min(target_sizes, key=lambda s: target_sizes[s])
    target_sizes[k] += 1
while sum(target_sizes.values()) > N:
    k = max(target_sizes, key=lambda s: target_sizes[s])
    target_sizes[k] -= 1

global_counts = Counter()
for _, _, _, labs, _ in items:
    for l in labs: 
        global_counts[l] += 1

label_targets = {s: {l: global_counts[l]*RATIOS[s] for l in all_labels} for s in RATIOS}

# ====== 3) Iterative stratification (라벨 수요 기반 배정) ======
# 라벨 → 해당 라벨 가진 샘플 인덱스
idxs_by_label = {l: set() for l in all_labels}
for i, it in enumerate(items):
    for l in it[3]:
        idxs_by_label[l].add(i)

assigned = {s: [] for s in RATIOS}
split_sizes = {s: 0 for s in RATIOS}
split_label_counts = {s: Counter() for s in RATIOS}
unassigned = set(range(N))

def cap_ok(s): 
    return split_sizes[s] < target_sizes[s]

def need(s, l):
    return label_targets[s][l] - split_label_counts[s][l]

def pick_label():
    # 남은 필요량 합(양수만) * 희소성 가중치 로 최대 라벨 선택
    best_l, best_score = None, -1e18
    for l in all_labels:
        if not (idxs_by_label[l] & unassigned):
            continue
        pos_need_sum = sum(max(0.0, need(s, l)) for s in RATIOS if cap_ok(s))
        rarity_w = 1.0 / max(1.0, global_counts[l])
        score = pos_need_sum * rarity_w
        if score > best_score:
            best_score, best_l = score, l
    return best_l

def assign_idx_to_best_split(i):
    labs = items[i][3]
    best_s, best_cost = None, 1e18
    for s in RATIOS:
        if not cap_ok(s): 
            continue
        # 라벨 오차 변화 + 사이즈 편차(가벼운 가중치)
        label_term = 0.0
        for l in labs:
            cur = split_label_counts[s][l]
            tgt = label_targets[s][l]
            label_term += abs((cur+1) - tgt) - abs(cur - tgt)
        size_term = abs((split_sizes[s]+1) - target_sizes[s]) - abs(split_sizes[s] - target_sizes[s])
        cost = label_term + 0.05*size_term
        if cost < best_cost:
            best_cost, best_s = cost, s
    if best_s is None:
        return False
    assigned[best_s].append(i)
    split_sizes[best_s] += 1
    for l in labs:
        split_label_counts[best_s][l] += 1
    unassigned.discard(i)
    for l in labs:
        idxs_by_label[l].discard(i)
    return True

# 메인 루프
while unassigned:
    l = pick_label()
    if l is None:
        break
    pool = list(idxs_by_label[l] & unassigned)
    random.shuffle(pool)
    splits_by_need = sorted([s for s in RATIOS if cap_ok(s)], key=lambda s: need(s, l), reverse=True)
    progressed = False
    for s in splits_by_need:
        best_i, best_cost = None, 1e18
        for i in pool:
            labs = items[i][3]
            label_term = 0.0
            for ll in labs:
                cur = split_label_counts[s][ll]
                tgt = label_targets[s][ll]
                label_term += abs((cur+1) - tgt) - abs(cur - tgt)
            size_term = abs((split_sizes[s]+1) - target_sizes[s]) - abs(split_sizes[s] - target_sizes[s])
            cost = label_term + 0.05*size_term
            if cost < best_cost:
                best_cost, best_i = cost, i
        if best_i is not None:
            assigned[s].append(best_i)
            split_sizes[s] += 1
            for ll in items[best_i][3]:
                split_label_counts[s][ll] += 1
            unassigned.discard(best_i)
            for ll in items[best_i][3]:
                idxs_by_label[ll].discard(best_i)
            pool.remove(best_i)
            progressed = True
            if not cap_ok(s):
                continue
    if not progressed:
        break

# 남은 샘플을 사이즈 목표 맞추며 라벨 오차 최소 split로 배정
if unassigned:
    rest = list(unassigned)
    random.shuffle(rest)
    for i in rest:
        assign_idx_to_best_split(i)

# ====== 4) 복사 ======
ensure_dirs()

def write_filtered_label(dst_lbl: Path, filtered_lines: list):
    """0/1만 남긴 filtered_lines로 새 라벨 저장. 비어있으면 라벨을 생성하지 않음."""
    if not filtered_lines:
        return
    dst_lbl.parent.mkdir(parents=True, exist_ok=True)
    with open(dst_lbl, "w") as f:
        for ln in filtered_lines:
            f.write(ln.rstrip() + "\n")

def copy_pair(idx, split):
    stem, img, lbl, labs, filtered = items[idx]
    # 이미지
    shutil.copy2(img, DST / "images" / split / img.name)
    # 라벨: 0/1만 남긴 라인으로 재생성 (bg나 빈 라벨은 생성 안 함)
    if filtered:
        dst_lbl = DST / "labels" / split / f"{stem}.txt"
        write_filtered_label(dst_lbl, filtered)
    # bg는 라벨 생성 안함 (YOLOv8이 자동으로 배경 처리)

for s, idxs in assigned.items():
    for i in idxs:
        copy_pair(i, s)

# ====== 5) data.yaml (2 클래스)
with open(DST / "data.yaml", "w") as f:
    f.write("train: images/train\n")
    f.write("val: images/val\n")
    f.write("test: images/test\n\n")
    f.write("nc: 2\n")
    f.write('names: ["Divot", "Fixed_Divot"]\n')  # ✅ class 2 제거

# ====== 6) 리포트 ======
def summarize():
    rep = []
    total = sum(len(v) for v in assigned.values())
    rep.append(f"Total images: {total}")
    rep.append(f"Targets: {target_sizes}")
    rep.append("Global label counts: " + ", ".join([f"{l}:{global_counts[l]}" for l in all_labels]))
    rep.append("Global label ratios: " + ", ".join([f"{l}:{global_counts[l]/total:.3f}" for l in all_labels]))
    for s in ["train","val","test"]:
        n = len(assigned[s])
        rep.append(f"\n[{s}] images: {n} (target {target_sizes[s]})")
        cnt = split_label_counts[s]
        rep.append("  label counts: " + ", ".join([f"{l}:{cnt[l]}" for l in all_labels]))
        rep.append("  label ratios: " + ", ".join([f"{l}:{cnt[l]/max(1,n):.3f}" for l in all_labels]))
        errs = []
        for l in all_labels:
            tgt = label_targets[s][l]
            err = cnt[l] - tgt
            rel = 0.0 if tgt==0 else err/max(1.0, tgt)
            errs.append(f"{l}:Δ{err:.1f}({rel:+.1%})")
        rep.append("  errors vs target: " + ", ".join(errs))
    return "\n".join(rep)

report = summarize()
print(report)
(Path(DST) / "split_report.txt").write_text(report, encoding="utf-8")
print(f"\n✅ Done. Split at: {DST}")
print("↳ data.yaml, split_report.txt saved.")


In [ ]:
from pathlib import Path
from collections import Counter

# 분할된 데이터셋 경로 (train/val/test 폴더가 들어있는 상위 폴더)
BASE = Path("/home/dw/ws_job_msislab/Golf_Project/data/good_data_3_erase_dieased_grass_aug_split")

splits = ["train", "val", "test"]

def count_instances(label_dir: Path):
    counter = Counter()
    for lbl in label_dir.rglob("*.txt"):
        with open(lbl, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5 and parts[0].isdigit():
                    cid = int(parts[0])
                    counter[cid] += 1
    return counter

# 전체 합
global_counter = Counter()

for split in splits:
    lbl_dir = BASE / "labels" / split
    cnt = count_instances(lbl_dir)
    global_counter.update(cnt)

    total = sum(cnt.values())
    print(f"\n📂 {split}")
    for cid in sorted(cnt):
        ratio = (cnt[cid] / total * 100) if total > 0 else 0
        print(f"  클래스 {cid}: {cnt[cid]}개 ({ratio:.2f}%)")
    print(f"  총 객체 수: {total}")

# 전체 분포
print("\n=== 전체 분포 ===")
grand_total = sum(global_counter.values())
for cid in sorted(global_counter):
    ratio = global_counter[cid] / grand_total * 100
    print(f"클래스 {cid}: {global_counter[cid]}개 ({ratio:.2f}%)")
print(f"총 객체 수: {grand_total}")


In [2]:
from ultralytics import YOLO

DATA_YAML = "/home/dw/ws_job_msislab/Golf_Project/data/good_data_3_erase_dieased_grass_aug_split/data.yaml"

# 1) 모델: s로 시작, 부족하면 m로 재학습
model = YOLO("yolov8s.pt")

results = model.train(
    data=DATA_YAML,
    epochs=300,
    patience=50,
    imgsz=1024,          # GPU 여유 없으면 1024로
    batch=-1,            # 자동 배치 (메모리 맞춰 자동 결정). 안되면 8~16 수동.
    device=0,
    workers=8,
    rect=True,
    cache=True,
    amp=True,            # 혼합정밀도

    # === 약한 온라인 증강 (오프라인에서 이미 많이 했으므로 가볍게) ===
    mosaic=0.10,         # 초반 소물체 리콜↑
    close_mosaic=15,     # 15에폭 이후 mosaic 자동 0
    mixup=0.0,
    copy_paste=0.0,
    hsv_h=0.01,
    hsv_s=0.2,
    hsv_v=0.2,
    degrees=0.0,         # 회전 금지
    translate=0.0,       # 위치 변형 금지
    scale=0.0,           # 스케일 변형 금지
    shear=0.0,           # 기하변형 금지
    erasing=0.0,

    # === 로스/최적화 쪽 튜닝 ===
    box=10.0,            # bbox 회귀 가중 ↑ (기본≈7.5). divot 로컬라이제이션 개선
    cls=0.5,             # 분류 로스 가중 (기본 수준)
    dfl=1.5,             # DFL 가중 (기본 수준)
    optimizer="AdamW",   # 안정적 수렴
    lr0=0.002,           # 초기 LR (s 모델 기준 안전 값)
    lrf=0.01,            # 최종 LR 비율 (cosine)
    momentum=0.9,
    weight_decay=0.01,

    # === 저장 경로 ===
    project="/home/dw/ws_job_msislab/Golf_Project/runs_yolo",
    name="good_data_3_erase_dieased_grass_aug",
    exist_ok=True

)

Ultralytics 8.3.175 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7932MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=10.0, cache=True, cfg=None, classes=None, close_mosaic=15, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/dw/ws_job_msislab/Golf_Project/data/good_data_3_erase_dieased_grass_aug_split/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.0, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.01, hsv_s=0.2, hsv_v=0.2, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.002, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.9, mosaic=0.1, multi_scale=False, name=good_data_3_erase_dieased_grass_aug, nbs=64, nms=False, opset=None, optimiz

train: Scanning /home/dw/ws_job_msislab/Golf_Project/data/good_data_3_erase_dieased_grass_aug_split/labels/train.cache... 1977 images, 218 backgrounds, 0 corrupt: 100%|██████████| 2195/2195 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
AutoBatch: Computing optimal batch size for imgsz=1024 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA GeForce RTX 4060 Ti) 7.75G total, 0.67G reserved, 0.16G allocated, 6.92G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output


    11136374       73.34         1.317         10.32         15.16      (1, 3, 1024, 1024)                    list
    11136374       146.7         2.068         14.33         29.32      (2, 3, 1024, 1024)                    list
    11136374       293.4         3.074         30.02          58.2      (4, 3, 1024, 1024)                    list
    11136374       586.7         5.551         72.34         118.7      (8, 3, 1024, 1024)                    list
CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 7.75 GiB of which 41.56 MiB is free. Including non-PyTorch memory, this process has 7.10 GiB memory in use. Of the allocated memory 6.84 GiB is allocated by PyTorch, and 69.77 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables

train: Scanning /home/dw/ws_job_msislab/Golf_Project/data/good_data_3_erase_dieased_grass_aug_split/labels/train.cache... 1977 images, 218 backgrounds, 0 corrupt: 100%|██████████| 2195/2195 [00:00<?, ?it/s]


...

    300/300      2.79G     0.1668     0.1114     0.7565          7       1024: 100%|██████████| 439/439 [00:44<00:00,  9.87it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:02<00:00, 13.37it/s]

                   all        338        984      0.913      0.793      0.891      0.752



300 epochs completed in 3.932 hours.
Optimizer stripped from /home/dw/ws_job_msislab/Golf_Project/runs_yolo/good_data_3_erase_dieased_grass_aug/weights/last.pt, 22.6MB
Optimizer stripped from /home/dw/ws_job_msislab/Golf_Project/runs_yolo/good_data_3_erase_dieased_grass_aug/weights/best.pt, 22.6MB

Validating /home/dw/ws_job_msislab/Golf_Project/runs_yolo/good_data_3_erase_dieased_grass_aug/weights/best.pt...
Ultralytics 8.3.175 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7932MiB)
Model summary (fused): 72 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:02<00:00, 14.04it/s]


                   all        338        984      0.888      0.808      0.893      0.757
                 Divot        289        575      0.897       0.79      0.887       0.75
           Fixed_Divot        195        409      0.879      0.826        0.9      0.763
Speed: 0.4ms preprocess, 5.5ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /home/dw/ws_job_msislab/Golf_Project/runs_yolo/good_data_3_erase_dieased_grass_aug
